In [35]:
from neo4j import GraphDatabase

uri = "neo4j+s://0d57704b.databases.neo4j.io"
driver = GraphDatabase.driver(uri, auth=("neo4j", "5yPsvhzqDCZYx2s08eS3GvGLPM33v32IaQp-jEG3CdM"))

In [36]:
def get_top_movies_by_language(language_name, k=10):
    """
    Get top k movies for a given language, sorted by vote count
    
    Args:
        language_name (str): Full language name (e.g., 'English', 'Japanese')
        k (int): Number of movies to return
    """
    with driver.session() as session:
        query = """
        MATCH (l:Language {name: $language_name})-[:IN_LANGUAGE]->(m:Movie)
        RETURN m.title as title,
               m.year as year,
               m.vote_count as votes,
               m.runtime as runtime,
               m.overview as overview
        ORDER BY m.vote_count DESC
        LIMIT $limit
        """
        
        result = session.run(query, language_name=language_name, limit=k)
        movies = []
        
        for record in result:
            movies.append({
                'title': record['title'],
                'year': record['year'],
                'votes': record['votes'],
                'runtime': record['runtime'],
                'overview': record['overview']
            })
            
        if not movies:
            print(f"No movies found for language: {language_name}")
            return
        
        print(f"\nTop {k} movies in {language_name}:")
        print("-" * 80)
        
        for i, movie in enumerate(movies, 1):
            print(f"{i}. {movie['title']} ({movie['year']})")
            print(f"   Votes: {movie['votes']:,} | Runtime: {movie['runtime']} min")
            print(f"   Overview: {movie['overview'][:200]}...")
            print("-" * 80)

# Example usage:
get_top_movies_by_language("Hindi", k=10)


Top 10 movies in Hindi:
--------------------------------------------------------------------------------
1. Dabba (2013)
   Votes: 221 | Runtime: 104 min
   Overview: A mistaken delivery in Mumbai's famously efficient lunchbox delivery system (Mumbai's Dabbawallahs) connects a young housewife to a stranger in the dusk of his life. They build a fantasy world togethe...
--------------------------------------------------------------------------------
2. Rang De Basanti (2006)
   Votes: 97 | Runtime: 157 min
   Overview: A young idealistic English filmmaker, Sue, arrives in India to make a film on Indian revolutionaries Bhagat Singh, Chandrashekhar Azad and their contemporaries and their fight for freedom from the Bri...
--------------------------------------------------------------------------------
3. Yeh Jawaani Hai Deewani (2013)
   Votes: 79 | Runtime: 159 min
   Overview: Yeh Jawaani Hai Deewani features two polar opposite characters, who at one point were classmates in Modern Schoo

In [37]:
def get_movies_by_director(director_name, k=10):
    """
    Get movies directed by a specific director, sorted by vote count
    
    Args:
        director_name (str): Full name of the director
        k (int): Number of movies to return
    """
    with driver.session() as session:
        query = """
        MATCH (d:Director {name: $director_name})-[:DIRECTED]->(m:Movie)
        RETURN m.title as title,
               m.year as year,
               m.vote_count as votes,
               m.runtime as runtime,
               m.overview as overview,
               m.rating as rating
        ORDER BY m.vote_count DESC
        LIMIT $limit
        """
        
        result = session.run(query, director_name=director_name, limit=k)
        movies = []
        
        for record in result:
            movies.append({
                'title': record['title'],
                'year': record['year'],
                'votes': record['votes'],
                'runtime': record['runtime'],
                'overview': record['overview'],
                'rating': record['rating']
            })
            
        if not movies:
            print(f"No movies found for director: {director_name}")
            return
        
        print(f"\nTop {k} movies directed by {director_name}:")
        print("-" * 80)
        
        for i, movie in enumerate(movies, 1):
            print(f"{i}. {movie['title']} ({movie['year']})")
            print(f"   Rating: {movie['rating']} | Votes: {movie['votes']:,} | Runtime: {movie['runtime']} min")
            print(f"   Overview: {movie['overview'][:200]}...")
            print("-" * 80)

# Example usage:
get_movies_by_director("Christopher Nolan")

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: rating)} {position: line: 8, column: 18, offset: 269} for query: '\n        MATCH (d:Director {name: $director_name})-[:DIRECTED]->(m:Movie)\n        RETURN m.title as title,\n               m.year as year,\n               m.vote_count as votes,\n               m.runtime as runtime,\n               m.overview as overview,\n               m.rating as rating\n        ORDER BY m.vote_count DESC\n        LIMIT $limit\n        '



Top 10 movies directed by Christopher Nolan:
--------------------------------------------------------------------------------
1. Inception (2010)
   Rating: None | Votes: 13,752 | Runtime: 148 min
   Overview: Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: "inc...
--------------------------------------------------------------------------------
2. The Dark Knight (2008)
   Rating: None | Votes: 12,002 | Runtime: 152 min
   Overview: Batman raises the stakes in his war on crime. With the help of Lt. Jim Gordon and District Attorney Harvey Dent, Batman sets out to dismantle the remaining criminal organizations that plague the stree...
--------------------------------------------------------------------------------
3. Interstellar (2014)
   Rating: None | Votes: 10,867 | Runtime: 169 min
   Overview: Interstellar chronicles the adventures of 

In [39]:
def get_movies_by_genre(genre_name, k=10):
    """
    Get top k movies for a given genre, sorted by vote count
    
    Args:
        genre_name (str): Name of the genre (e.g., 'Action', 'Drama', 'Comedy')
        k (int): Number of movies to return
    """
    with driver.session() as session:
        query = """
        MATCH (m:Movie)-[:HAS_GENRE]->(g:Genre {name: $genre_name})
        WHERE m.vote_count IS NOT NULL
        RETURN m.title as title,
               m.year as year,
               m.vote_count as votes,
               m.runtime as runtime,
               m.overview as overview,
               m.rating as rating
        ORDER BY m.vote_count DESC, m.rating DESC
        LIMIT $limit
        """
        
        result = session.run(query, genre_name=genre_name, limit=k)
        movies = []
        
        for record in result:
            movies.append({
                'title': record['title'],
                'year': record['year'],
                'votes': record['votes'],
                'runtime': record['runtime'],
                'overview': record['overview'],
                'rating': record['rating']
            })
            
        if not movies:
            print(f"No movies found for genre: {genre_name}")
            return
        
        print(f"\nTop {k} {genre_name} movies:")
        print("-" * 80)
        
        for i, movie in enumerate(movies, 1):
            print(f"{i}. {movie['title']} ({movie['year']})")
            print(f"   Rating: {movie['rating']} | Votes: {movie['votes']:,} | Runtime: {movie['runtime']} min")
            print(f"   Overview: {movie['overview'][:200]}...")
            print("-" * 80)

# Example usage:
get_movies_by_genre("Science Fiction", k=10)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: rating)} {position: line: 9, column: 18, offset: 303} for query: '\n        MATCH (m:Movie)-[:HAS_GENRE]->(g:Genre {name: $genre_name})\n        WHERE m.vote_count IS NOT NULL\n        RETURN m.title as title,\n               m.year as year,\n               m.vote_count as votes,\n               m.runtime as runtime,\n               m.overview as overview,\n               m.rating as rating\n        ORDER BY m.vote_count DESC, m.rating DESC\n        LIMIT $limit\n        '



Top 10 Science Fiction movies:
--------------------------------------------------------------------------------
1. Inception (2010)
   Rating: None | Votes: 13,752 | Runtime: 148 min
   Overview: Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: "inc...
--------------------------------------------------------------------------------
2. Avatar (2009)
   Rating: None | Votes: 11,800 | Runtime: 162 min
   Overview: In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization....
--------------------------------------------------------------------------------
3. The Avengers (2012)
   Rating: None | Votes: 11,776 | Runtime: 143 min
   Overview: When an unexpected enemy emerges and threatens global safety and security, Nick Fury, dire